# Estimates of Value-at-Risk for shares of a Bucharest Stock Exchange-traded company, 2015-2019

**Author:** Nicholas VJ Alexander


In [1]:
!pip -q install nbconvert
!pip -q install ipython
!pip -q install yfinance
!pip -q install matplotlib
!pip -q install cryptography
import requests, getpass, base64, os
import pandas as pd
import numpy as np
import yfinance as yf
from IPython.display import Markdown as md
from datetime import datetime
from scipy.stats import norm, t
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

In [2]:

currtime = datetime.now().strftime('%Y-%m-%d %H:%M UTC')
md(f"**Most recently updated:** {currtime}")


**Most recently updated:** 2024-01-20 17:48 UTC


**Contact:** https://www.linkedin.com/in/nvalexander

---

**Abstract:**

This working paper tests several methods of estimation for Value-at-Risk (VaR), comparing their performance with the actual findings over the subsequent 12 months.

---

**Disclaimer:**

This file is work in progress. As long as this disclaimer is included, its results are not to be assumed as correct.

---


The European Union is at the forefront of global banking reforms aimed at mitigating risks in the private banking system. Additionally, the core objective of the European Banking Authority is a transparent banking environment. To these goals, Directive 2013/36/EU of the European Parliament and of the Council, and its implementation, Commission Implementing Regulation (eu) 2019/912, require adherence to Basel III rules across the EU member states. The latter's Annex 4 requires Credit Institutions disclose, every year, "Own funds requirements for market risk" using "standardized methods".

The strictness of these regulations is merely a façade. The current Basel framework approach, in the section "MAR33. Internal models approach: capital requirements calculation", states that "supervisors may permit banks to use models based on either historical simulation, Monte Carlo simulation, or other appropriate analytical methods". Hence, in this working paper, I will test several methods for estimating Value-at-Risk (VaR) in a hypothetical portfolio comprising a single type of equity, namely a company traded on the Bucharest Stock Exchange.


## Methods

Closing values for the interval 2015-2019 are obtained through a HTTP scraper written by the author. As it not clear whether scraping bvb.ro data is in agreement with its rules, I protected a trivial section of the scraper with a password in the code posted on Github.

In [3]:
def key_from_password(password: str, salt: bytes):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=base64.urlsafe_b64decode(salt),
        iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
    return key

def decrypt_message(encrypted_message, password, salt):
    key = key_from_password(password, salt)
    fernet = Fernet(key)
    try:
        decrypted_message = fernet.decrypt(encrypted_message.encode()).decode()
        return decrypted_message
    except:
        return None

def fetch_bvb_data(ticker, startdate, enddate):
    start_timestamp = int(datetime.strptime(startdate, '%Y-%m-%d').timestamp())
    end_timestamp = int(datetime.strptime(enddate, '%Y-%m-%d').timestamp())
    url = f'https://wapi.bvb.ro/api/history?symbol={ticker}&dt=DAILY&p=day&ajust=0&from={start_timestamp}&to={end_timestamp}'
    referrer_url = decrypt_message(
        'gAAAAABlq-CfDvJGetCCbdadT9aEdRwD9AjQNL9CcYh8aINR_uiBOOrD_58SDsTN9pO0IE4twO2ohut3SMEHpFRSfskGWv6N048H3y1yKQWWhyb-kpFPuLhFA6UkkTvLcKduTBwxXFG4m8LeP-RPkYcxi9TrHjX3uTkrFcBKIyA54XtgWcMvLr22XDeMaWLCg2bN85W5UadJ', 
        password,
        'RYGlNyoOQupNEt4l0f8IdQ=='
        )
    if not referrer_url:
        print("Incorrect password, cannot decrypt the URL.")
        return pd.DataFrame()
    headers = {'Referer': referrer_url}
    response = requests.get(url, headers=headers)
    del referrer_url
    del headers
    if response.status_code != 200:
        return pd.DataFrame()  # Return an empty DataFrame in case of unsuccessful response
    data = response.json()
    if data.get('s') != 'ok':
        return pd.DataFrame()  # Return an empty DataFrame if status is not 'ok'
    return pd.DataFrame({
        'Date': [datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d') for ts in data['t']],
        'Open': data['o'],
        'High': data['h'],
        'Low': data['l'],
        'Close': data['c'],
        'Volume': data['v']
    })

password = getpass.getpass("Enter your BVB fetcher password: ")
ticker = 'EL'
startdate = '2016-01-01'
enddate = '2021-01-01'
df = fetch_bvb_data(ticker, startdate, enddate)
#print(df)

del password

In [4]:
df['DateTime'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='DateTime')
df['Daily_Return'] = df['Close'].pct_change()

train_data = df[(df['DateTime'] >= '2015-01-01') & (df['DateTime'] < '2019-01-01')]['Daily_Return'].dropna()
test_data = df[df['DateTime'] >= '2019-01-01']['Daily_Return'].dropna()

confidence_level = 0.95

# Historical Simulation method
VaR_HS = np.percentile(train_data, (1 - confidence_level) * 100)
ES_HS = train_data[train_data <= VaR_HS].mean()

# Normal Distribution method
mean_return, std_dev = norm.fit(train_data)
VaR_Normal = norm.ppf(1 - confidence_level, mean_return, std_dev)
ES_Normal = mean_return + std_dev * norm.pdf(norm.ppf(1 - confidence_level)) / (1 - confidence_level)

# t-Distribution method
degrees_of_freedom, loc, scale = t.fit(train_data)
VaR_t = t.ppf(1 - confidence_level, degrees_of_freedom, loc, scale)
ES_t = (degrees_of_freedom / (degrees_of_freedom - 1) * loc + scale * t.pdf(t.ppf(1 - confidence_level, degrees_of_freedom, loc, scale), degrees_of_freedom, loc, scale) / (1 - confidence_level)) if degrees_of_freedom > 1 else np.nan

# Output the results
print(f"VaR (Historical Simulation): {VaR_HS}")
print(f"ES (Historical Simulation): {ES_HS}")
print(f"VaR (Normal Distribution): {VaR_Normal}")
print(f"ES (Normal Distribution): {ES_Normal}")
print(f"VaR (t-Distribution): {VaR_t}")
print(f"ES (t-Distribution): {ES_t}")


VaR (Historical Simulation): -0.01904761904761898
ES (Historical Simulation): -0.030985270396360773
VaR (Normal Distribution): -0.02109922627245241
ES (Normal Distribution): 0.02600319014511007
VaR (t-Distribution): -0.018319758488147077
ES (t-Distribution): 0.8544695687918541


In [5]:
exceedances = test_data[test_data < VaR_HS].count()
expected_exceedances = len(test_data) * (1 - confidence_level)
print(f"Observed Exceedances: {exceedances}")
print(f"Expected Exceedances: {expected_exceedances}")

Observed Exceedances: 36
Expected Exceedances: 24.900000000000023
